In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from geopy import distance

pd.set_option('max.columns',50)

In [2]:
# set path to github directory
os.chdir('..')

# set path to raw data folder
github_path = Path(os.getcwd())
data_path = github_path / 'data' / 'raw'

In [3]:
# load data frames
airbnb = pd.read_csv(data_path / 'train.csv')
city = pd.read_csv(data_path / 'worldcities.csv')

In [4]:
airbnb.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [5]:
city.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37977000.0,1392685764
1,Jakarta,Jakarta,-6.2146,106.8451,Indonesia,ID,IDN,Jakarta,primary,34540000.0,1360771077
2,Delhi,Delhi,28.6600,77.2300,India,IN,IND,Delhi,admin,29617000.0,1356872604
3,Mumbai,Mumbai,18.9667,72.8333,India,IN,IND,Mahārāshtra,admin,23355000.0,1356226629
4,Manila,Manila,14.5958,120.9772,Philippines,PH,PHL,Manila,primary,23088000.0,1608618140


In [6]:
airbnb.city.unique()

array(['NYC', 'SF', 'DC', 'LA', 'Chicago', 'Boston'], dtype=object)

In [7]:
# select relevant cities
city_df = city[city['id'].astype('str').isin(['1840034016', '1840020491', '1840000494', 
                                '1840006060', '1840000455', '1840021543'])]
city_df = city_df.rename(columns={'id':'city_id', 'city':'city_name'})
city_df['city_id'] = city_df['city_id'].astype('str')

In [8]:
bnb_city_id = {'NYC':'1840034016', 'SF':'1840021543', 'DC':'1840006060', 
 'LA':'1840020491', 'Chicago':'1840000494', 'Boston':'1840000455'}

airbnb['city_id'] = airbnb['city'].map(bnb_city_id)

In [9]:
train = pd.merge(airbnb, city_df, how='left', on='city_id')

In [10]:
def dist_city_center(prop_lat, prop_lon, city_lat, city_lon):
    prop_coords = (prop_lat, prop_lon)
    city_coords = (city_lat, city_lon)
    dist = distance.great_circle(prop_coords, city_coords).miles
    return dist

In [12]:
train['cc_dist'] = train.apply(lambda x: dist_city_center(x.latitude, x.longitude, x.lat, x.lng), axis=1)

In [13]:
# def num_close_airbnbs(y, train, miles):
#     df = train.copy()
#     df['rel_dist'] = df.apply(lambda x: dist_city_center(y.latitude, y.longitude, x.latitude, x.longitude), axis=1)
#     num_close = df[df['rel_dist']<=miles]
#     return len(num_close)-1

# train['bnb_within_1mi'] = train.apply(lambda y: num_close_airbnbs(y, train, 1), axis=1)

In [14]:
def haversine_np(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)

    All args must be of equal length.    

    """
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
#     m = km * 0.621371
    
    return km

In [15]:
def num_close_airbnbs(y, df, dist_thresh):
    df['cur_lat'] = y.latitude
    df['cur_lon'] = y.longitude
    df['rel_dist'] = haversine_np(df['cur_lon'], df['cur_lat'], df['longitude'], df['latitude'])
    num_close = df[df['rel_dist']<=dist_thresh]
    return len(num_close)-1

In [16]:
# takes ~10 mins to run
train['num_bnb_1km'] = train.apply(lambda y: num_close_airbnbs(y, train, 1), axis=1)

# train['num_bnb_3mi'] = train.apply(lambda y: num_close_airbnbs(y, train, 3), axis=1)
# train['num_bnb_5mi'] = train.apply(lambda y: num_close_airbnbs(y, train, 5), axis=1)
# train['num_bnb_10mi'] = train.apply(lambda y: num_close_airbnbs(y, train, 10), axis=1)

train = train.drop(columns=['cur_lat', 'cur_lon', 'rel_dist'])

In [17]:
train.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,city_id,city_name,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,cc_dist,num_bnb_1km
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",2016-06-18,t,t,NaN,2012-03-26,f,2016-07-18,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0,1840034016,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0,3.498384,272
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,2017-08-05,t,f,100%,2017-06-19,t,2017-09-23,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0,1840034016,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0,5.991584,1400
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,2017-04-30,t,t,100%,2016-10-25,t,2017-09-14,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0,1840034016,New York,New York,40.6943,-73.9249,United States,US,USA,New York,NaN,18713220.0,7.925215,891
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,2015-04-19,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117.0,2.0,2.0,1840021543,San Francisco,San Francisco,37.7562,-122.4430,United States,US,USA,California,NaN,3592294.0,1.256540,666
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",2015-05-12,t,t,100%,2015-03-01,t,2017-01-22,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0,1840006060,Washington,Washington,38.9047,-77.0163,United States,US,USA,District of Columbia,primary,5379184.0,1.748733,727
